# Multilayer Perceptron

Multilayer Perceptron (MLP) is a type of neural network that can be used for regression and classification.

This version of the workbook includes mini-batching which was added in the 1.14 release.

In [1]:
%load_ext sql

/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/fmcquillan/anaconda/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [2]:
# Greenplum Database 5.4.0 on GCP (demo machine)
%sql postgresql://gpadmin@35.184.253.255:5432/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

# Greenplum Database 4.3.10.0
#%sql postgresql://gpdbchina@10.194.10.68:61000/madlib

u'Connected: gpadmin@madlib'

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.14-dev, git revision: rc/1.13-rc1-66-g4cced1b, cmake configuration time: Mon Apr 23 16:26:17 UTC 2018, build type: release, build system: Linux-2.6.32-696.20.1.el6.x86_64, C compiler: gcc 4.4.7, C++ compiler: g++ 4.4.7"


# Classification without Mini-Batching

# 1.  Create input table for classification

In [4]:
%%sql 
DROP TABLE IF EXISTS iris_data;

CREATE TABLE iris_data(
    id serial,
    attributes numeric[],
    class_text varchar,
    class integer,
    state varchar
);

INSERT INTO iris_data(id, attributes, class_text, class, state) VALUES
(1,ARRAY[5.0,3.2,1.2,0.2],'Iris_setosa',1,'Alaska'),
(2,ARRAY[5.5,3.5,1.3,0.2],'Iris_setosa',1,'Alaska'),
(3,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Alaska'),
(4,ARRAY[4.4,3.0,1.3,0.2],'Iris_setosa',1,'Alaska'),
(5,ARRAY[5.1,3.4,1.5,0.2],'Iris_setosa',1,'Alaska'),
(6,ARRAY[5.0,3.5,1.3,0.3],'Iris_setosa',1,'Alaska'),
(7,ARRAY[4.5,2.3,1.3,0.3],'Iris_setosa',1,'Alaska'),
(8,ARRAY[4.4,3.2,1.3,0.2],'Iris_setosa',1,'Alaska'),
(9,ARRAY[5.0,3.5,1.6,0.6],'Iris_setosa',1,'Alaska'),
(10,ARRAY[5.1,3.8,1.9,0.4],'Iris_setosa',1,'Alaska'),
(11,ARRAY[4.8,3.0,1.4,0.3],'Iris_setosa',1,'Alaska'),
(12,ARRAY[5.1,3.8,1.6,0.2],'Iris_setosa',1,'Alaska'),
(13,ARRAY[5.7,2.8,4.5,1.3],'Iris_versicolor',2,'Alaska'),
(14,ARRAY[6.3,3.3,4.7,1.6],'Iris_versicolor',2,'Alaska'),
(15,ARRAY[4.9,2.4,3.3,1.0],'Iris_versicolor',2,'Alaska'),
(16,ARRAY[6.6,2.9,4.6,1.3],'Iris_versicolor',2,'Alaska'),
(17,ARRAY[5.2,2.7,3.9,1.4],'Iris_versicolor',2,'Alaska'),
(18,ARRAY[5.0,2.0,3.5,1.0],'Iris_versicolor',2,'Alaska'),
(19,ARRAY[5.9,3.0,4.2,1.5],'Iris_versicolor',2,'Alaska'),
(20,ARRAY[6.0,2.2,4.0,1.0],'Iris_versicolor',2,'Alaska'),
(21,ARRAY[6.1,2.9,4.7,1.4],'Iris_versicolor',2,'Alaska'),
(22,ARRAY[5.6,2.9,3.6,1.3],'Iris_versicolor',2,'Alaska'),
(23,ARRAY[6.7,3.1,4.4,1.4],'Iris_versicolor',2,'Alaska'),
(24,ARRAY[5.6,3.0,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(25,ARRAY[5.8,2.7,4.1,1.0],'Iris_versicolor',2,'Alaska'),
(26,ARRAY[6.2,2.2,4.5,1.5],'Iris_versicolor',2,'Alaska'),
(27,ARRAY[5.6,2.5,3.9,1.1],'Iris_versicolor',2,'Alaska'),
(28,ARRAY[5.0,3.4,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(29,ARRAY[4.4,2.9,1.4,0.2],'Iris_setosa',1,'Tennessee'),
(30,ARRAY[4.9,3.1,1.5,0.1],'Iris_setosa',1,'Tennessee'),
(31,ARRAY[5.4,3.7,1.5,0.2],'Iris_setosa',1,'Tennessee'),
(32,ARRAY[4.8,3.4,1.6,0.2],'Iris_setosa',1,'Tennessee'),
(33,ARRAY[4.8,3.0,1.4,0.1],'Iris_setosa',1,'Tennessee'),
(34,ARRAY[4.3,3.0,1.1,0.1],'Iris_setosa',1,'Tennessee'),
(35,ARRAY[5.8,4.0,1.2,0.2],'Iris_setosa',1,'Tennessee'),
(36,ARRAY[5.7,4.4,1.5,0.4],'Iris_setosa',1,'Tennessee'),
(37,ARRAY[5.4,3.9,1.3,0.4],'Iris_setosa',1,'Tennessee'),
(38,ARRAY[6.0,2.9,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(39,ARRAY[5.7,2.6,3.5,1.0],'Iris_versicolor',2,'Tennessee'),
(40,ARRAY[5.5,2.4,3.8,1.1],'Iris_versicolor',2,'Tennessee'),
(41,ARRAY[5.5,2.4,3.7,1.0],'Iris_versicolor',2,'Tennessee'),
(42,ARRAY[5.8,2.7,3.9,1.2],'Iris_versicolor',2,'Tennessee'),
(43,ARRAY[6.0,2.7,5.1,1.6],'Iris_versicolor',2,'Tennessee'),
(44,ARRAY[5.4,3.0,4.5,1.5],'Iris_versicolor',2,'Tennessee'),
(45,ARRAY[6.0,3.4,4.5,1.6],'Iris_versicolor',2,'Tennessee'),
(46,ARRAY[6.7,3.1,4.7,1.5],'Iris_versicolor',2,'Tennessee'),
(47,ARRAY[6.3,2.3,4.4,1.3],'Iris_versicolor',2,'Tennessee'),
(48,ARRAY[5.6,3.0,4.1,1.3],'Iris_versicolor',2,'Tennessee'),
(49,ARRAY[5.5,2.5,4.0,1.3],'Iris_versicolor',2,'Tennessee'),
(50,ARRAY[5.5,2.6,4.4,1.2],'Iris_versicolor',2,'Tennessee'),
(51,ARRAY[6.1,3.0,4.6,1.4],'Iris_versicolor',2,'Tennessee'),
(52,ARRAY[5.8,2.6,4.0,1.2],'Iris_versicolor',2,'Tennessee');

SELECT * FROM iris_data ORDER BY id;

Done.
Done.
52 rows affected.
52 rows affected.


id,attributes,class_text,class,state
1,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


#  2. Classification model

Generate a multilayer perceptron with a single hidden layer of 5 units. Use the attributes column as the independent variables, and use the class column as the classification. Set the tolerance to 0 so that 500 iterations will be run. Use a hyperbolic tangent activation function. The model will be written to mlp_model.

In [5]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=500,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model:

In [6]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[-0.133201972097394, -0.118636963016892, 0.259474215534915, -0.528433916977086, -0.429596986146934, 0.0280938067215663, -0.144466640003299, 0.286988287730496, -0.357379109879214, -0.66607966047993, -0.362686864800844, -0.445850222259492, 0.346304179999463, -0.522045614901381, -0.374150944327096, -0.0495674317777646, -0.0832467361276034, 0.279692771132311, -0.517927322791807, -0.2809929864922, -0.183324281085865, -0.243554165368264, 0.349093795759817, -0.713886988614578, -0.432209713264906, -0.450592821509468, 0.43151079821727, 0.472695001337664, 0.612226054527927, 0.490963383058475, 0.724327654407535, -0.103407228476278, -0.851088815290572, -0.837299697527914, -0.899486162064411, -0.486098700715332, -1.08911290004112]",0.00734471737812,500


View the model summary table:

In [7]:
%%sql
SELECT * FROM mlp_model_summary;

1 rows affected.


source_table,independent_varname,dependent_varname,dependent_vartype,tolerance,learning_rate_init,learning_rate_policy,n_iterations,n_tries,layer_sizes,activation,is_classification,classes,weights,grouping_col
iris_data,attributes,class_text,character varying,0.0,0.003,constant,500,1,"[4, 5, 2]",tanh,True,"[u'Iris_setosa', u'Iris_versicolor']",1,NULL


View the standardization table:

In [8]:
%%sql
SELECT * FROM mlp_model_standardization;

1 rows affected.


mean,std
"[5.45961538461539, 2.99807692307692, 3.025, 0.851923076923077]","[0.598799958694505, 0.498262513685689, 1.41840579525043, 0.550346179381454]"


# 3. Prediction for classification

Now let's use the model to predict. In the following example we will use the training data set for prediction as well, which is not usual but serves to show the syntax. The prediction is in the estimated_class_text column with the actual value in the class_text column.

In [9]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model',         -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (id) ORDER BY id;

Done.
1 rows affected.
52 rows affected.


id,estimated_class_text,attributes,class_text,class,state
1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


Count the misclassifications:

In [10]:
%%sql
SELECT COUNT(*) FROM mlp_prediction JOIN iris_data USING (id) 
WHERE mlp_prediction.estimated_class_text != iris_data.class_text;

1 rows affected.


count
0


# Classification with Mini-Batching

# 1. Call mini-batch preprocessor

Use the same data set as above.

In [11]:
%%sql
DROP TABLE IF EXISTS iris_data_packed, iris_data_packed_summary, iris_data_packed_standardization;

SELECT madlib.minibatch_preprocessor('iris_data',         -- Source table
                                     'iris_data_packed',  -- Output table
                                     'class_text',        -- Dependent variable
                                     'attributes'        -- Independent variables
                                     );

SELECT * FROM iris_data_packed ORDER BY __id__;

Done.
1 rows affected.
2 rows affected.


__id__,dependent_varname,independent_varname
0,"[[0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0]]","[[0.0674425821070736, -0.798930106411465, 0.969398182526215, 0.632469046062059], [0.902445979718656, -0.196837852308928, 1.03989986852812, 1.17758048907675], [-0.433559456459875, -0.598232688377286, 0.616889752516682, 0.995876674738521], [-1.10156217454914, 0.00385956572525086, -1.14565239753098, -1.36627291165848], [-0.767560815504508, 0.405254401793609, -1.28665576953479, -1.18456909732025], [0.568444620674023, -0.798930106411465, 0.687391438518589, 0.632469046062059], [-0.767560815504508, 1.00734665589615, -1.00464902552717, -0.457753839967327], [1.4034480182856, -1.401022360514, 0.969398182526215, 0.81417286040029], [-0.767560815504508, -2.00311461461654, 0.334883008509056, 0.269061417385597], [-1.10156217454914, 0.00385956572525086, -1.14565239753098, -1.00286528298202], [0.568444620674023, -0.598232688377286, 0.616889752516682, 0.632469046062059], [1.06944665924097, 0.00385956572525086, 1.11040155453003, 0.995876674738521], [0.234443261629389, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [0.234443261629389, -0.999627524445644, 0.616889752516682, 0.450765231723828], [0.902445979718656, 0.806649237861967, 1.03989986852812, 1.35928430341498], [0.0674425821070736, -0.999627524445644, 0.687391438518589, 0.81417286040029], [1.23644733876329, -1.60171977854818, 1.03989986852812, 1.17758048907675], [0.0674425821070736, -1.20032494247982, 0.475886380512869, 0.269061417385597], [-1.93656557216072, 0.00385956572525086, -1.3571574555367, -1.36627291165848], [-0.0995580974152422, 1.4087414919645, -1.07515071152907, -1.18456909732025], [2.07145073637487, 0.20455698375943, 0.969398182526215, 0.995876674738521], [-1.76956489263841, 0.00385956572525086, -1.21615408353289, -1.18456909732025], [-0.767560815504508, 1.00734665589615, -1.21615408353289, -1.00286528298202], [-0.0995580974152422, 0.00385956572525086, 1.03989986852812, 1.17758048907675], [0.234443261629389, 0.00385956572525086, 0.757893124520495, 0.81417286040029]]"
1,"[[0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0]]","[[0.401443941151707, -0.397535270343108, 1.03989986852812, 0.81417286040029], [0.568444620674023, -0.598232688377286, 0.757893124520495, 0.269061417385597], [-0.0995580974152422, 1.81013632803286, -1.21615408353289, -0.821161468643789], [0.902445979718656, -1.60171977854818, 0.687391438518589, 0.269061417385597], [-0.600560135982193, 1.60943890999868, -0.793143967521448, -0.821161468643789], [-1.76956489263841, -0.196837852308928, -1.14565239753098, -1.18456909732025], [0.0674425821070736, 1.00734665589615, -1.21615408353289, -1.18456909732025], [-1.76956489263841, 0.405254401793609, -1.21615408353289, -1.18456909732025], [0.401443941151707, -0.798930106411465, 0.334883008509056, 0.269061417385597], [-0.767560815504508, 0.806649237861967, -1.07515071152907, -1.18456909732025], [0.401443941151707, 2.81362341820376, -1.07515071152907, -0.821161468643789], [-1.60256421311609, -1.401022360514, -1.21615408353289, -1.00286528298202], [0.568444620674023, 2.01083374606704, -1.28665576953479, -1.18456909732025], [0.902445979718656, -0.598232688377286, 1.46290998453956, 1.35928430341498], [0.73544530019634, 0.00385956572525086, 0.828394810522402, 1.17758048907675], [-0.934561495026824, 0.20455698375943, -1.07515071152907, -1.36627291165848], [-0.600560135982193, 0.80664

# 2.  Classification model with mini-batching
Use similar parameters as before:

In [12]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;
-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data_packed',      -- Ouput table from mini-batch preprocessor
    'mlp_model',             -- Destination table
    'independent_varname',   -- Hardcode to this, from table iris_data_packed
    'dependent_varname',     -- Hardcode to this, from table iris_data_packed
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.1,
    n_iterations=500,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model:

In [13]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[-0.0309749079415527, -0.238585231565989, 0.516459799389647, -0.46368579959856, -0.673732925023756, 0.0778619338393474, 0.197305073296416, -0.249692782186095, 0.439028430336739, 0.626376493360924, -0.0882970823387187, -0.0797679288590299, 0.397818124348168, -0.123298748840092, -0.207996015482563, -0.225365582875435, -0.311523866687928, 0.323570097811814, -0.369573237920434, -0.364962791069138, 0.233034700676389, 0.220908774672402, -0.292167680244236, 0.695958014086603, 0.619175545678452, -0.164066120012632, 0.951372990648697, -0.819801614646947, 0.290676073162377, 0.785234769097514, -1.04842776221521, 0.172123879960554, -0.94312904330421, 0.658608739158027, -0.264472191827765, -0.37546065538845, 0.923108343066969]",0.00603272550899,500


# 3. Prediction for classification

Now predict using model we built using mini-batching.  As before we will use the training data set for prediction as well, which is not usual but serves to show the syntax. The prediction is in the estimated_class_text column with the actual value in the class_text column.  Note that the prediction function is exactly the same whether you use mini batching or not:

In [14]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model',         -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (id) ORDER BY id;

Done.
1 rows affected.
52 rows affected.


id,estimated_class_text,attributes,class_text,class,state
1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1,Alaska
2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1,Alaska
4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1,Alaska
6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1,Alaska
8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1,Alaska
9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1,Alaska
10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1,Alaska


Count the missclassifications:

In [15]:
%%sql
SELECT COUNT(*) FROM mlp_prediction JOIN iris_data USING (id) 
WHERE mlp_prediction.estimated_class_text != iris_data.class_text;

1 rows affected.


count
0


# Classification with Other Parameters

# 1.  Classification with n_tries
Now, use the n_tries optimizer parameter to learn and choose the best model among n_tries number of models learned by the algorithm. Run only for 50 iterations and choose the best model from this short run.  Note we are not using mini-batching here.

In [16]:
%%sql
DROP TABLE IF EXISTS mlp_model, mlp_model_summary, mlp_model_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=50,
    tolerance=0,
    n_tries=3',       -- Optimizer params, with n_tries
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model:

In [17]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[-0.0446467911836253, -0.149462234984348, 0.260067826326458, -0.294188184731176, -0.268351877896166, -0.120110546273018, -0.297879579672679, 0.382373630421642, -0.266724261199798, -0.360580936078716, 0.170001566774959, 0.311443942865912, -0.127869679146657, 0.71725588398749, 0.0204255812415505, -0.0207367800502355, -0.244003181464991, 0.259923947073304, -0.147612797018657, -0.0437507820443381, 0.138239045063333, -0.0229267847182451, 0.0518025470588523, -0.0822488994012624, -0.209397161217787, -0.202597315852859, 0.256827874320701, 0.76757733949347, -0.603135413374952, 0.338954401220572, 0.209390818536744, -0.0470922620589311, -0.508204648805407, -0.304491233281693, 0.498440490225775, -0.222426925536799, -0.0304855053963594]",0.17540579361,50


# 2. Classification with warm start
Next, use the warm_start parameter to start learning a new model, using the coefficients already present in mlp_model. Note that we must not drop the mlp_model table, and cannot use the n_tries parameter if warm_start is used.

In [18]:
%%sql
SELECT madlib.mlp_classification(
    'iris_data',      -- Source table
    'mlp_model',      -- Destination table
    'attributes',     -- Input features
    'class_text',     -- Label
    ARRAY[5],         -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=450,
    tolerance=0',     -- Optimizer params
    'tanh',           -- Activation function
    NULL,             -- Default weight (1)
    TRUE,             -- Warm start
    FALSE             -- Not verbose
);

1 rows affected.


mlp_classification
""


View the classification model with warm_start.  The algorithm learnt three different models starting with a different set of initial weights for the coefficients, and chose the best model among them.

In [20]:
%%sql
SELECT * FROM mlp_model;

1 rows affected.


coeff,loss,num_iterations
"[-0.127784817258819, -0.233746623762672, 0.394625199676419, -0.481587130109332, -0.456675304339317, -0.201370388740204, -0.372071524491049, 0.511262213429843, -0.452599139454441, -0.547819373547151, 0.265255263660732, 0.349770334465604, -0.321275204753894, 0.900215284083308, 0.20936476478716, -0.0929222168773207, -0.33614872946673, 0.395416045681193, -0.346532187443256, -0.245570331896227, 0.0869280165676589, -0.085792720665623, 0.14589940250351, -0.206740358326462, -0.333431675468536, -0.282524742816705, 0.598946910391331, 1.14773394599332, -0.974968285323165, 0.60984929495699, 0.385632758478045, 0.0328351649048052, -0.850323684875331, -0.684647839782326, 0.870273362174163, -0.493321819272987, -0.206727445337447]",0.00795191031737,450


# Classification with grouping


# 1. Train classification model
Group the training data by state, and learn a different model for each state.  Note we are not using mini-batching in this example.

In [21]:
%%sql
DROP TABLE IF EXISTS mlp_model_group, mlp_model_group_summary, mlp_model_group_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_classification(
    'iris_data',        -- Source table
    'mlp_model_group',  -- Destination table
    'attributes',       -- Input features
    'class_text',       -- Label
    ARRAY[5],           -- Number of units per layer
    'learning_rate_init=0.003,
    n_iterations=500,
    tolerance=0',       -- Optimizer params
    'tanh',             -- Activation function
    NULL,               -- Default weight (1)
    FALSE,              -- No warm start
    FALSE,              -- Not verbose
    'state'             -- Grouping column
);

Done.
1 rows affected.
1 rows affected.


mlp_classification
""


View the classification model with grouping.  A separate model is learnt for each state, and the result table displays the name of the state (grouping column) associated with the model.

In [22]:
%%sql
SELECT * FROM mlp_model_group ORDER BY state;

2 rows affected.


state,coeff,loss,num_iterations
Alaska,"[-0.0602983524314911, -0.348000521728888, 0.394786119329522, -0.401466348624141, -0.483836378109328, -0.0116116214169814, -0.261757500137133, -0.177276593694436, -0.149567186850256, -0.478916102587215, -0.154989439330733, -0.424543179702658, 0.593667303714511, -0.658947585764815, -0.66620563993319, 0.0335741204532086, -0.10910071520199, 0.259191513208525, -0.11349669294211, -0.462696102914078, 0.11501914526655, -0.34634121690547, 0.0404009390190409, 0.264575597890908, -0.330097704271105, -0.299726852345694, 0.685209763008927, -0.0353315838063156, 1.47889648577834, 0.33752349612434, 0.302058531620214, 0.0211223855121644, -0.789253782440321, -0.588837662232531, -1.10169743483528, -0.416122295684595, 0.0697611636673644]",0.0250663545493,500
Tennessee,"[0.211734589925836, 0.237653307089714, -0.4368759218487, 0.272265833128647, 0.500689582503131, 0.0823341615119298, 0.239668052413637, -0.389976095154171, 0.396908441604442, 0.449524308353627, -0.110173944369731, 0.133997548962495, -0.0652154933191128, 0.171692785095863, -0.122719997373715, -0.166863320817755, -0.575312019280707, 0.565176983538667, -0.285953308887656, -0.447173010310337, -0.00873901095690873, 0.452027381647434, -0.398219155478815, 0.444139643487796, 0.306936159629734, -0.304330794092475, -0.816608753146611, -0.356367496615788, -0.321759537145298, 0.713117819272002, -0.724991150162783, 0.107521607759632, 0.514478952311261, 0.93505102853775, -0.201719502644605, -1.04609591565432, 0.474907617688023]",0.0180946110007,500


# 2. Prediction for classification using grouping
Prediction based on grouping using the state column:

In [23]:
%%sql
DROP TABLE IF EXISTS mlp_prediction;

SELECT madlib.mlp_predict(
         'mlp_model_group',   -- Model table
         'iris_data',         -- Test data table
         'id',                -- Id column in test table
         'mlp_prediction',    -- Output table for predictions
         'response'           -- Output classes, not probabilities
     );

SELECT * FROM mlp_prediction JOIN iris_data USING (state,id) ORDER BY state, id;

Done.
1 rows affected.
52 rows affected.


state,id,estimated_class_text,attributes,class_text,class
Alaska,1,Iris_setosa,"[Decimal('5.0'), Decimal('3.2'), Decimal('1.2'), Decimal('0.2')]",Iris_setosa,1
Alaska,2,Iris_setosa,"[Decimal('5.5'), Decimal('3.5'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,3,Iris_setosa,"[Decimal('4.9'), Decimal('3.1'), Decimal('1.5'), Decimal('0.1')]",Iris_setosa,1
Alaska,4,Iris_setosa,"[Decimal('4.4'), Decimal('3.0'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,5,Iris_setosa,"[Decimal('5.1'), Decimal('3.4'), Decimal('1.5'), Decimal('0.2')]",Iris_setosa,1
Alaska,6,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1
Alaska,7,Iris_setosa,"[Decimal('4.5'), Decimal('2.3'), Decimal('1.3'), Decimal('0.3')]",Iris_setosa,1
Alaska,8,Iris_setosa,"[Decimal('4.4'), Decimal('3.2'), Decimal('1.3'), Decimal('0.2')]",Iris_setosa,1
Alaska,9,Iris_setosa,"[Decimal('5.0'), Decimal('3.5'), Decimal('1.6'), Decimal('0.6')]",Iris_setosa,1
Alaska,10,Iris_setosa,"[Decimal('5.1'), Decimal('3.8'), Decimal('1.9'), Decimal('0.4')]",Iris_setosa,1


# Regression without Mini-Batching

# 1. Create input table for regression
This dataset contains housing prices data, where zipcode can be used as a grouping column.

In [24]:
%%sql
DROP TABLE IF EXISTS lin_housing;

CREATE TABLE lin_housing (id serial, 
                          x numeric[], 
                          zipcode int, 
                          y float8);

INSERT INTO lin_housing(id, x, zipcode, y) VALUES
(1,ARRAY[1,0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98],94016,24.00),
(2,ARRAY[1,0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14],94016,21.60),
(3,ARRAY[1,0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03],94016,34.70),
(4,ARRAY[1,0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94],94016,33.40),
(5,ARRAY[1,0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6.0622,3,222.0,18.70,396.90,5.33],94016,36.20),
(6,ARRAY[1,0.02985,0.00,2.180,0,0.4580,6.4300,58.70,6.0622,3,222.0,18.70,394.12,5.21],94016,28.70),
(7,ARRAY[1,0.08829,12.50,7.870,0,0.5240,6.0120,66.60,5.5605,5,311.0,15.20,395.60,12.43],94016,22.90),
(8,ARRAY[1,0.14455,12.50,7.870,0,0.5240,6.1720,96.10,5.9505,5,311.0,15.20,396.90,19.15],94016,27.10),
(9,ARRAY[1,0.21124,12.50,7.870,0,0.5240,5.6310,100.00,6.0821,5,311.0,15.20,386.63,29.93],94016,16.50),
(10,ARRAY[1,0.17004,12.50,7.870,0,0.5240,6.0040,85.90,6.5921,5,311.0,15.20,386.71,17.10],94016,18.90),
(11,ARRAY[1,0.22489,12.50,7.870,0,0.5240,6.3770,94.30,6.3467,5,311.0,15.20,392.52,20.45],94016,15.00),
(12,ARRAY[1,0.11747,12.50,7.870,0,0.5240,6.0090,82.90,6.2267,5,311.0,15.20,396.90,13.27],20001,18.90),
(13,ARRAY[1,0.09378,12.50,7.870,0,0.5240,5.8890,39.00,5.4509,5,311.0,15.20,390.50,15.71],20001,21.70),
(14,ARRAY[1,0.62976,0.00,8.140,0,0.5380,5.9490,61.80,4.7075,4,307.0,21.00,396.90,8.26],20001,20.40),
(15,ARRAY[1,0.63796,0.00,8.140,0,0.5380,6.0960,84.50,4.4619,4,307.0,21.00,380.02,10.26],20001,18.20),
(16,ARRAY[1,0.62739,0.00,8.140,0,0.5380,5.8340,56.50,4.4986,4,307.0,21.00,395.62,8.47],20001,19.90),
(17,ARRAY[1,1.05393,0.00,8.140,0,0.5380,5.9350,29.30,4.4986,4,307.0,21.00,386.85,6.58],20001, 23.10),
(18,ARRAY[1,0.78420,0.00,8.140,0,0.5380,5.9900,81.70,4.2579,4,307.0,21.00,386.75,14.67],20001,17.50),
(19,ARRAY[1,0.80271,0.00,8.140,0,0.5380,5.4560,36.60,3.7965,4,307.0,21.00,288.99,11.69],20001,20.20),
(20,ARRAY[1,0.72580,0.00,8.140,0,0.5380,5.7270,69.50,3.7965,4,307.0,21.00,390.95,11.28],20001,18.20);

SELECT * FROM lin_housing ORDER BY id;

Done.
Done.
20 rows affected.
20 rows affected.


id,x,zipcode,y
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9


# 2. Train regression model
Now train a regression model using a multilayer perceptron with 2 hidden layers of 25 nodes each.

In [25]:
%%sql
DROP TABLE IF EXISTS mlp_regress, mlp_regress_summary, mlp_regress_standardization;

SELECT setseed(0);
SELECT madlib.mlp_regression(
    'lin_housing',    -- Source table
    'mlp_regress',    -- Desination table
    'x',              -- Input features
    'y',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE             -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


View the regression model:

In [26]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([-0.00165149748390649, 0.132139021327922, -0.186931326684385, 0.123880190836735, 0.104351061983398, 0.0265969340172693, 0.0415840909614712, -0.361427063587442, 0.124342105681705, -0.125069509157226, -0.130187964871171, -0.0308686114144794, -0.124245868416245, 0.196500234142793, -0.385041862689158, 0.274045746702148, -0.0996100504411175, 0.118180197597448, -0.436571306035844, -0.151197845495664, -0.0145604290299056, -0.0386994354301253, 0.579748368878552, -0.138259510227382, 0.254060814325093, -0.113900682011834, -0.095610684769881, 0.444574495272539, -0.220793289865075, -0.330594530145564, -0.0286773575403892, -0.0950296850543527, -0.106822824073575, -0.0787793998347875, -0.0533164763405062, -0.0337896750194041, -0.0528523165081386, 0.0883221621854532, -0.0280909169439096, 0.00543596706036995, 0.0520974544278104, 0.107310756812307, 0.0059808359577235, 0.00762469190655167, -0.0260097103880477, -0.0993245101082391, -0.0340602079538714, -0.253059951008932, -0.078152460974249, 0.0160527692110396, 0.104626803151125, 0.0731767409150244, -0.0977506996232556, 0.0306013007963316, -0.0799056085650217, 0.113581427803493, 0.0280823058129219, -0.194001683541512, 0.00159593201227422, 0.0447890261599231, -0.0213618844257925, 0.140733985253583, -0.00176180552398824, 0.0574421490104688, -0.126454052368941, -0.0371241100819721, 0.0281946711370571, 0.0118797232904518, -0.186255383107036, 0.0260398240893396, 0.0266878980152484, 0.0492660518151597, -0.0447844646478541, 0.00640468502330932, 0.0689301083676493, 0.0023163192129433, 0.0826587872241641, -0.240381970137377, -0.0262637832711324, 0.253342101008011, 0.0347226800252171, 0.166935863591063, 0.220920649157035, 0.136122559881938, -0.742754389354896, 0.226413150642853, 0.161186990747007, 0.0971610030537311, 0.424033084705019, 0.102789181715678, -0.07648866137, -0.0759513736200082, -0.324270118695433, -0.0249593168173169, -0.134485277976096, 0.044519927273744, -0.184112402492504, 0.109805299567884, 0.470500275372347, -0.508114733223871, 0.167745399724765, -0.195325774175035, -0.0468591776299705, 0.311258836485316, -0.0203766624727048, -0.048869273979322, 0.111151873420583, 0.029424745221864, -0.0860095259202891, 0.187072208789783, -0.037475385784422, 0.0296145203927202, 0.384478472048028, -0.0561643450445722, -0.0904395663940604, 0.0110501987673447, 0.151639605692911, 0.0130493593930094, 0.122119619643692, -0.0274360576910505, 0.162558074307589, -0.130911214032721, -0.234074241547839, 0.0642471483519262, -0.00758462229260503, 0.119769161030989, 0.00976447539294623, 0.0371127678657603, -0.00488636692254843, -0.0014601935799613, -0.0150338117698949, -0.00557108182478719, -0.0769346538054085, -0.0659364549269095, -0.0643956965632451, -0.0539464967956825, -0.0299766932357753, -0.0685532619557751, 0.0881980660260438, 0.000631007325861735, 0.00862430881548466, -0.172456681129521, 0.176611307774089, -0.0157561668461728, -0.0777729918627719, -0.025218664400875, -0.0543716656853277, -0.0423929702182809, -0.0617118230557383, -0.109672402319021, 0.132631511879611, -0.0278890850940609, 0.0355800972802887, -0.288472482444074, -0.130618559576356, -0.00340382739120247, -0.0111389141931639, 0.278905011424022, -0.15158181616387, 0.121893152949638, -0.0192898723733432, -0.0126747903024488, 0.165422444833035, -0.127142108329151, -0.205741291414303, 0.085638902301927, 0.054523182302195, 0.1452572771159, -0.193440681477945, 0.0427524400731329, -0.00686361769784724, 0.188339212272079, 0.00112378102044158, -0.0918924966876459, -0.0756983592078081, 0.129561701898265, 0.0350753327405751, 0.206131249989464, -0.179485994680923, 0.0167276828671063, 0.0602020464218143, 0.00671731942065409, -0.21857225940444, 0.230285492476464, 0.191415378903611, -0.132433994358682, -0.048446737630388, -0.0533765951798685, 0.197143604808261, -0.221310870555434, 0.231132058976161, 0.0878263603653009, -0.178114240862251, 0.0808645825370069, -0.120331956220005, 0.0514672452405245, -0.0259316500290739, 0.0566869272380828, -0.0519307128703512, -

# 3.  Prediction for regression

In [27]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress',               -- Model table
         'lin_housing',               -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT *, ABS(y-estimated_y) as abs_diff FROM lin_housing JOIN mlp_regress_prediction USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,x,zipcode,y,estimated_y,abs_diff
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0,24.0406839111,0.0406839111205
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6,22.0464413965,0.446441396508
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7,34.3653806332,0.334619366803
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4,33.8190945738,0.419094573809
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2,35.2488768942,0.951123105849
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7,28.0800098953,0.619990104711
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9,23.3812616893,0.481261689301
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1,23.8835353381,3.21646466189
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5,17.5815891385,1.08158913855
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9,17.2047146397,1.69528536027


RMS error:

In [28]:
%%sql
SELECT SQRT(SUM(ABS(y-estimated_y))/COUNT(y)) as rms_error FROM lin_housing 
JOIN mlp_regress_prediction USING (id);

1 rows affected.


rms_error
0.87366405429


# Regression with Mini-Batching

# 1. Call mini-batch preprocessor

Call mini-batch preprocessor using the same data set as above:

In [31]:
%%sql
DROP TABLE IF EXISTS lin_housing_packed, lin_housing_packed_summary, lin_housing_packed_standardization;

SELECT madlib.minibatch_preprocessor('lin_housing',         -- Source table
                                     'lin_housing_packed',  -- Output table
                                     'y',                   -- Dependent variable
                                     'x'                   -- Independent variables
                                     );

SELECT * FROM lin_housing_packed ORDER BY __id__;

Done.
1 rows affected.
2 rows affected.


__id__,dependent_varname,independent_varname
0,"[[27.1], [20.2], [20.4], [24.0], [33.4], [23.1], [21.7], [17.5], [16.5], [18.2]]","[[0.0, -0.549415882738228, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.0422863607397744, 1.43847656712523, 0.814588992812889, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.421245698698072, 1.17246185773607], [0.0, 1.45215368098917, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -1.56757776072358, -1.54685150207432, -1.59359191400827, 0.130188910980857, 0.53579457077381, 1.18392008678395, -4.26790463620074, 0.0225058416026117], [0.0, 0.926185138314665, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.517342201796184, -0.282477261001572, -0.575090147010091, 0.130188910980857, 0.53579457077381, 1.18392008678395, 0.421245698698072, -0.506227286458749], [0.0, -0.969795366137461, 1.93914963595328, -1.94020778590522, 0.0, 0.776989868951453, 0.816222625563962, -0.11188708561874, -1.26545770038152, -2.47358930863628, 0.215482599115554, -1.08062504565515, 0.421245698698072, -1.01183797451743], [0.0, -0.890573160085591, -0.803851813725231, -1.9969773137395, 0.0, -1.8347407829862, 1.71733751689518, -1.08525455692078, 0.939470054694005, -0.737737162224855, -1.93934339203999, 0.270156261413788, 0.322604504552763, -1.32630315855392], [0.0, 2.21615378096425, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.547166335315421, -1.91311864333746, -0.808641210814613, 0.130188910980857, 0.53579457077381, 1.18392008678395, -0.0154697203152993, -0.765198614488803], [0.0, -0.703815552651988, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.645159916878625, -1.42643490768644, 0.256034126959662, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.143138367187069, 0.642187233674527], [0.0, 1.395861817879, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.430000096489847, 0.715977000797943, -1.07774480611259, 0.130188910980857, 0.53579457077381, 1.18392008678395, -0.0198151473701598, 0.481871649655922], [0.0, -0.346600952926358, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -1.19477609173313, 1.63415353300554, 0.961718336980792, 0.998114984186569, 0.652271651376812, -1.12035390762777, -0.0250296598359913, 2.83419454592891], [0.0, 1.21825810066482, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.990267747601218, 0.103859312659547, -1.59359191400827, 0.130188910980857, 0.53579457077381, 1.18392008678395, 0.162692788933936, -0.040695494404723]]"
1,"[[15.0], [36.2], [18.2], [34.7], [18.9], [21.6], [19.9], [18.9], [28.7], [22.9]]","[[0.0, -0.305089125187087, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, 0.394424165791903, 1.34816412133432, 1.25754223110562, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.230915993695229, 1.37285633775932], [0.0, -0.779023428109291, -0.803851813725231, -1.9969773137395, 0.0, -1.8347407829862, 2.03475150934991, -0.663796476563195, 0.939470054694005, -0.737737162224855, -1.93934339203999, 0.270156261413788, 0.421245698698072, -0.957885614511164], [0.0, 0.951122646553642, -0.803851813725231, 0.605687193124138, 0.0, 0.776989868951453, -0.2041887998442, 0.856463027583804, -0.849671962569644, 0.130188910980857, 0.53579457077381, 1.18392008678395, -0.3122623881622, -0.19792808642297], [0.0, -0.90602225055559, -0.803851813725231, 0.138430310180499, 0.0, -1.47562781834477, 2.11570272890212, -0.317598767698037, -0.284856273560335, -1.60566323543057, -1.35695798902498, -0.0874034963397545, 0.244386817565293, -1.15828009453442], [0.0, -0.631770483117677, 1.10101030410707, 0.487781250699107, 0.0, 0.319937004862364, -0.389524486713741, 0.776185297991884, 1.12338173490388, 0.998114984186569, 0.652271651376812, -1.12035390762777, 0.421245698698072, 0.266062209630877], [0.0, -0.905961427364764, -0.803851813725231, 0.138430310180499, 0.0, -1.47562781834477, 0.488157156852361, 0.575490974012082, -0.284856273560335, -1.60566323543057, -1.35695

# 2. Train regression model with mini-batching

In [32]:
%%sql
DROP TABLE IF EXISTS mlp_regress, mlp_regress_summary, mlp_regress_standardization;

SELECT setseed(0);
SELECT madlib.mlp_regression(
    'lin_housing_packed',    -- Source table
    'mlp_regress',           -- Desination table
    'independent_varname',   -- Hardcode to this, from table lin_housing_packed
    'dependent_varname',     -- Hardcode to this, from table lin_housing_packed
    ARRAY[25,25],            -- Number of units per layer
    'learning_rate_init=0.01,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',            -- Optimizer params
    'tanh',                  -- Activation function
    NULL,                    -- Default weight (1)
    FALSE,                   -- No warm start
    FALSE                    -- Not verbose
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


View model:

In [33]:
%%sql
SELECT * FROM mlp_regress;

1 rows affected.


[([-0.492851797062635, -0.0655092176758542, 0.0532167369903224, -0.0875365402142294, -0.144437001047355, -0.0367315815155944, 0.0586750956687185, -0.339979136883618, 0.225669807632894, -0.125858682318646, -0.448313998007996, -0.0868629227727297, 0.116162519865637, 0.136739783550816, 0.00121018186350306, 0.451352594623066, 0.038779951379162, -0.258693195620151, -0.194825416032824, -0.119583914091555, 0.1696354080044, -0.194433634560518, -0.436788165281276, 0.0704016826335396, -0.33405254754443, -0.0621983763594281, -0.243558420037681, -0.0892691088626092, -0.0348116598268225, 0.202660360576285, -0.151740022117008, -0.10480775392765, 0.0242363032610604, 0.0627818032716188, 0.104951202561425, 0.0354380010531103, -0.0741723559263544, 0.335246783268708, -0.0167908800966605, 0.0768723390108929, 0.201053212823101, -0.078542902311081, -0.209771648321082, -0.0900467032842112, -0.205495766385638, 0.516820326094735, -0.0403032837119905, -0.162337889508108, 0.112697570288577, -0.21969007730436, -0.108118656298869, -0.0212223509948013, -0.169504750174137, -0.0791815373219589, -0.393704985871069, 0.0331468313256545, -0.0791509930544647, -0.134367325860622, -0.0702971862767748, -0.174273818704306, -0.486989146052734, 0.0309776715713334, 0.13088212938004, 0.106904769631687, 0.188380720020085, -0.0308463133992448, 0.233340213355918, 0.337686255139687, -0.23432710482101, 0.443083657498776, 0.102923760394993, 0.166829226610179, -0.100071572522283, -0.0831928407977715, -0.0302131247557863, -0.00632842252020234, -0.145197254693425, 0.169058389833703, 0.174961237275871, 0.0155277150206453, -0.00172275551373838, 0.145694563618931, 0.0776132217908398, 0.0438430777686606, 0.0202694703176214, 0.0597908069804118, 0.34231753841072, 0.0316948752235019, -0.00097881549514401, -0.027477936996186, -0.147987679649919, 0.102174184128885, 0.215716908821837, 0.22718524708351, -0.0506132566389185, 0.0361620087079233, 0.0239943632623688, -0.0720164616275768, -0.210565302864795, 0.298676041978209, 0.0133787664942917, 0.0112470924379862, -0.142133526050282, -0.00849059305340421, 0.024521081663294, 0.0650952450753051, -0.0060659966982427, -0.138331623041698, -0.052324904086114, 0.0118348326964541, -0.0247365388627715, -0.0559942978844471, 0.00259175158415178, 0.140971524379278, -0.173697328753411, -0.0699856627490467, -0.0978486912848851, -0.0547324774579751, 0.0225739806371161, -0.017743323166603, 0.325263254838093, -0.0181194467881349, 0.0553520977163807, 0.253093046375837, -0.045539134957923, -0.0319371758133356, 0.0524690341049846, 0.530362045679259, -0.247942512647728, -0.0150692020364579, 0.268844202500643, 0.1571754001218, -0.376456111681553, 0.178057734976216, 0.139183301564763, 0.463215550664215, -0.0118303036621897, 0.193053060091224, 0.0791697972853251, 0.079369594685579, -0.0308045142714486, -0.0927124835207268, 0.441805743523436, -0.0846193414625826, 0.0788220068590674, 0.458808226343599, 0.189665411375778, -0.154017177596334, -0.0403656532490576, 0.095048843824123, -0.0578269493143806, -0.0983813326626066, 0.247351385378735, -0.222394202267384, 0.224990199782296, -0.11863096237437, 0.0969811158764001, -0.0985376837505285, 0.135721724676058, 0.255148112967556, -0.205782471371329, -0.0388790147212811, 0.142120991401283, -0.271217781142384, 0.178538103702461, 0.174528849524213, 0.127567569194257, 0.0372483314806806, 0.0173380736690387, 0.0148557413020189, 0.0666342390776691, -0.0900296786455517, 0.00922637284192545, 0.0269807970313625, 0.202016772108043, 0.117479767545541, -0.054702180399501, 0.10811404197183, 0.0880267157972175, -0.203803356430199, -0.0726377122209998, -0.112998009290298, 0.117346216682721, 0.18092115413922, 0.117607531494795, -0.0894730244978053, 0.384576617512255, 0.121897190218937, -0.302402326707247, 0.320814418038711, -0.0393847112361163, 0.356561772733997, -0.0123482987005726, -0.158783634585537, -0.0142315961300611, -0.639422218380823, 0.0937238840053451, 0.104818562903042, 0.48834403497965, 0.0494231373642286, -0.0302052716747808, 

# 3.  Prediction for regression

In [34]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress',               -- Model table
         'lin_housing',               -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT *, ABS(y-estimated_y) as abs_diff FROM lin_housing JOIN mlp_regress_prediction USING (id) ORDER BY id;

Done.
1 rows affected.
20 rows affected.


id,x,zipcode,y,estimated_y,abs_diff
1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",94016,24.0,24.032506572,0.0325065720214
2,"[Decimal('1'), Decimal('0.02731'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('6.4210'), Decimal('78.90'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('396.90'), Decimal('9.14')]",94016,21.6,21.8285237023,0.228523702268
3,"[Decimal('1'), Decimal('0.02729'), Decimal('0.00'), Decimal('7.070'), Decimal('0'), Decimal('0.4690'), Decimal('7.1850'), Decimal('61.10'), Decimal('4.9671'), Decimal('2'), Decimal('242.0'), Decimal('17.80'), Decimal('392.83'), Decimal('4.03')]",94016,34.7,34.6505207158,0.0494792841856
4,"[Decimal('1'), Decimal('0.03237'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.9980'), Decimal('45.80'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.63'), Decimal('2.94')]",94016,33.4,34.6778421762,1.27784217615
5,"[Decimal('1'), Decimal('0.06905'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('7.1470'), Decimal('54.20'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('396.90'), Decimal('5.33')]",94016,36.2,34.8360873764,1.36391262362
6,"[Decimal('1'), Decimal('0.02985'), Decimal('0.00'), Decimal('2.180'), Decimal('0'), Decimal('0.4580'), Decimal('6.4300'), Decimal('58.70'), Decimal('6.0622'), Decimal('3'), Decimal('222.0'), Decimal('18.70'), Decimal('394.12'), Decimal('5.21')]",94016,28.7,29.091370941,0.391370941016
7,"[Decimal('1'), Decimal('0.08829'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0120'), Decimal('66.60'), Decimal('5.5605'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('395.60'), Decimal('12.43')]",94016,22.9,23.0972669471,0.197266947148
8,"[Decimal('1'), Decimal('0.14455'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.1720'), Decimal('96.10'), Decimal('5.9505'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('19.15')]",94016,27.1,22.6939590741,4.40604092593
9,"[Decimal('1'), Decimal('0.21124'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.6310'), Decimal('100.00'), Decimal('6.0821'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.63'), Decimal('29.93')]",94016,16.5,15.4530514983,1.04694850173
10,"[Decimal('1'), Decimal('0.17004'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0040'), Decimal('85.90'), Decimal('6.5921'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('386.71'), Decimal('17.10')]",94016,18.9,15.8089429045,3.09105709547


RMS error:

In [35]:
%%sql
SELECT SQRT(SUM(ABS(y-estimated_y))/COUNT(y)) as rms_error FROM lin_housing 
JOIN mlp_regress_prediction USING (id);

1 rows affected.


rms_error
0.946026526865


# Regression with Grouping and Mini-Batching

# 1. Call mini-batch preprocessor
To use grouping and mini-batching, we must first re-run the preprocessor and specify grouping by zipcode (i.e., grouping must exactly match between mini-batch preprocessor and training function).

In [37]:
%%sql
DROP TABLE IF EXISTS lin_housing_packed, lin_housing_packed_summary, lin_housing_packed_standardization;

SELECT madlib.minibatch_preprocessor('lin_housing',         -- Source table
                                     'lin_housing_packed',  -- Output table
                                     'y',                   -- Dependent variable
                                     'x',                   -- Independent variables
                                     'zipcode'              -- Group by zipcode
                                     );

Done.
1 rows affected.


minibatch_preprocessor
""


# 2. Train regression model with grouping and mini-batching

In [38]:
%%sql
DROP TABLE IF EXISTS mlp_regress_group, mlp_regress_group_summary, mlp_regress_group_standardization;

-- Set seed so results are reproducible
SELECT setseed(0);

SELECT madlib.mlp_regression(
    'lin_housing_packed',    -- Source table
    'mlp_regress_group',    -- Desination table
    'independent_varname',              -- Input features
    'dependent_varname',              -- Dependent variable
    ARRAY[25,25],     -- Number of units per layer
    'learning_rate_init=0.001,
    n_iterations=500,
    lambda=0.001,
    tolerance=0',     -- Optimizer params
    'relu',           -- Activation function
    NULL,             -- Default weight (1)
    FALSE,            -- No warm start
    FALSE,            -- Not verbose
    'zipcode'         -- Grouping column
);

Done.
1 rows affected.
1 rows affected.


mlp_regression
""


View regression model with grouping:

In [39]:
%%sql
SELECT * FROM mlp_regress_group;

2 rows affected.


[(20001, [0.0603364269242927, 0.0764793549189194, -0.108391628771024, -0.0743912442720077, -0.0492814828929634, 0.0791888685258264, -0.0515512769207878, -0.0890154637626305, 0.0058961329746587, 0.0464971844475253, 0.135298246326767, 0.104500250527497, 0.105133588002041, 0.087882319687266, 0.261082474325745, -0.107553771922257, 0.0561670539683779, -0.0746522925560119, -0.0781958108948173, -0.0138506125549393, 0.104464348393354, -0.0698417493402519, 0.0799491258032308, -0.0258159320427502, -0.0342962791059317, 0.0567384628075608, -0.0737885772104546, 0.100583532669291, -0.106297044144801, -0.0530811333731039, 0.00960130602217303, -0.138611425925033, -0.00698058895461731, 0.079438177996555, -0.0358629729746381, 0.170155964149661, 0.171896064942071, 0.139714310056236, -0.119212967692761, -0.00205983830579382, -0.0389666138573766, -0.0620320926870361, -0.110724910757807, -0.0620614530737006, -0.195085468094129, -0.0187507813238056, 0.0389721061378674, 0.0797276387238253, 0.00446566081490591, -0.0919660573812058, -0.109079542197302, 0.0969575695384233, -0.0950013204509865, 0.158904941476577, 0.0712599112779968, 0.057701250906392, 0.0249140407433563, -0.0258019187583696, -0.0230588857155985, -0.0822930840304578, 0.472263499746989, 0.131846167816621, -0.176129544061221, 0.198586990330106, -0.037961096836781, 0.10563442459229, 0.0313085042977208, 0.0611970822020777, -0.136194364123924, 0.249123221971839, 0.0396840279151741, 0.108209687888578, -0.13469725748017, 0.171624350387516, -0.0878290429038543, 0.168226211485713, 0.083379357015154, 0.0171976000391356, -0.250454458779842, 0.200521499152446, -0.096265564818464, 0.0521168550571336, 0.0505184103566385, 0.0183717407950384, -0.127605862844599, -0.236258714306671, -0.272278689387431, 0.0473041440384629, 0.062673495418003, -0.075426896613676, 0.0485276810581846, -0.0180895844583542, -0.0724041994148226, -0.021903580118533, 0.0921308995890434, -0.0624005847101709, -0.104218625065382, -0.000478186411747838, 0.123256857910068, -0.0453917363432246, -0.0297200970667076, -0.106510906860852, -0.0294714820762954, -0.0690553096823886, -0.0494827727181978, 0.230787353203985, 0.0948438940804443, -0.249848619313454, -0.0519272585836548, 0.0635204503961411, 0.132382848194705, -0.121581649164011, -0.171617201428889, -0.200725539892757, 0.166511242991231, 0.087088527817574, 0.0991211083944472, -0.120982408566436, -0.110967035453519, 0.137910020750146, 0.011885787102527, -0.0606492105025423, 0.0139028062716279, 0.0804285947744426, 0.136549011861238, -0.0966664653342348, 0.0340419972481633, 0.0599790288803703, 0.114540664635236, -0.0520451221349719, -0.058872278622554, -0.147306470695437, 0.0562781137792344, 0.137231424735616, -0.00754910212847942, -0.013007540026806, 0.0288833867733776, -0.0935517518312294, 0.0467012897794239, 0.0966630160721845, -0.149287629362169, -0.000345530856509732, 0.0408837611462025, -0.0461675039454597, -0.0816110769112574, -0.0246873533056136, -0.0844630716591621, -0.0828144197744474, -0.0121958091895885, 0.0188567090565303, -0.0757903253590844, 0.141855526162179, 0.00282851682050236, 0.13784552058269, 0.053077072630025, -0.0772495407841775, -0.138923574991049, 0.0828749501371559, -0.060743032429587, 0.134170972994127, -0.178435004954784, -0.0940908364010028, 0.0439278685981206, -0.108269691389867, 0.0158783962020923, 0.339780339409362, -0.117636659503358, 0.173732198083604, -0.0655292713850715, 0.307683230789887, 0.131919779887376, 0.329617698752933, -0.0131788954367759, 0.00823280713291514, -0.170854743588048, -0.133059596935371, -0.203209093643862, 0.196832656138185, -0.067240404176451, 0.0334276804489045, 0.762747087901245, -0.111723983168362, 0.306039563393085, -0.472567405368045, 0.200225893276948, -0.0690235171581461, 0.370726666453025, -0.0178042833593276, -0.135056278300954, -0.184485043680839, -0.528597340780404, -0.519664843751239, 0.216737761179594, 0.0715566234270278, -0.331505379688511, -0.0120359192403956, -0.116504235636453, -0.0446509896154925, -0.016536432251

# 3. Prediction for regression with grouping and mini-batching

In [41]:
%%sql
DROP TABLE IF EXISTS mlp_regress_prediction;

SELECT madlib.mlp_predict(
         'mlp_regress_group',         -- Model table
         'lin_housing',             -- Test data table
         'id',                        -- Id column in test table
         'mlp_regress_prediction',    -- Output table for predictions
         'response'                   -- Output values, not probabilities
     );

SELECT * FROM lin_housing JOIN mlp_regress_prediction USING (zipcode, id) ORDER BY zipcode, id;

Done.
1 rows affected.
20 rows affected.


zipcode,id,x,y,estimated_y
20001,12,"[Decimal('1'), Decimal('0.11747'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('6.0090'), Decimal('82.90'), Decimal('6.2267'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('396.90'), Decimal('13.27')]",18.9,19.4406744128
20001,13,"[Decimal('1'), Decimal('0.09378'), Decimal('12.50'), Decimal('7.870'), Decimal('0'), Decimal('0.5240'), Decimal('5.8890'), Decimal('39.00'), Decimal('5.4509'), Decimal('5'), Decimal('311.0'), Decimal('15.20'), Decimal('390.50'), Decimal('15.71')]",21.7,21.1947012227
20001,14,"[Decimal('1'), Decimal('0.62976'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9490'), Decimal('61.80'), Decimal('4.7075'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('396.90'), Decimal('8.26')]",20.4,19.7660404614
20001,15,"[Decimal('1'), Decimal('0.63796'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('6.0960'), Decimal('84.50'), Decimal('4.4619'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('380.02'), Decimal('10.26')]",18.2,18.5003059969
20001,16,"[Decimal('1'), Decimal('0.62739'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.8340'), Decimal('56.50'), Decimal('4.4986'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('395.62'), Decimal('8.47')]",19.9,19.6864005592
20001,17,"[Decimal('1'), Decimal('1.05393'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9350'), Decimal('29.30'), Decimal('4.4986'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('386.85'), Decimal('6.58')]",23.1,23.4059404493
20001,18,"[Decimal('1'), Decimal('0.78420'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.9900'), Decimal('81.70'), Decimal('4.2579'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('386.75'), Decimal('14.67')]",17.5,17.1866426198
20001,19,"[Decimal('1'), Decimal('0.80271'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.4560'), Decimal('36.60'), Decimal('3.7965'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('288.99'), Decimal('11.69')]",20.2,20.0768583509
20001,20,"[Decimal('1'), Decimal('0.72580'), Decimal('0.00'), Decimal('8.140'), Decimal('0'), Decimal('0.5380'), Decimal('5.7270'), Decimal('69.50'), Decimal('3.7965'), Decimal('4'), Decimal('307.0'), Decimal('21.00'), Decimal('390.95'), Decimal('11.28')]",18.2,18.7792961651
94016,1,"[Decimal('1'), Decimal('0.00632'), Decimal('18.00'), Decimal('2.310'), Decimal('0'), Decimal('0.5380'), Decimal('6.5750'), Decimal('65.20'), Decimal('4.0900'), Decimal('1'), Decimal('296.0'), Decimal('15.30'), Decimal('396.90'), Decimal('4.98')]",24.0,23.9604850851
